In [2]:
import streamlit as st
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image

In [3]:
# App title
st.title("Rice Type Classification App")

# Extract zip file once
#zip_path = r"C:\Users\puram\Downloads\RICE TYPES.zip"
extract_to = r"C:\Users\puram\Downloads\RICE TYPES\RICE TYPES"

In [5]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

#Load training data (you can also cache this if needed)
train_data = datagen.flow_from_directory(
    extract_to,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    extract_to,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 49 images belonging to 6 classes.
Found 11 images belonging to 6 classes.


In [3]:
# Build model
model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),
    layers.Conv2D(16, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
#Train model (use fewer epochs for speed)
model.fit(train_data, validation_data=val_data, epochs=20)

#Save model (only once)
model.save("rice_model.keras")

#Upload and predict
st.header("Upload a rice grain image for prediction")
uploaded_file = st.file_uploader("Choose an image", type=['jpg', 'png', 'jpeg'])

if uploaded_file is not None:
    img = Image.open(uploaded_file).resize((128, 128))
    st.image(img, caption="Uploaded Image", use_column_width=True)

    img_array = np.expand_dims(np.array(img) / 255.0, axis=0)

    #Load saved model
    loaded_model = load_model("rice_model.keras")

    prediction = loaded_model.predict(img_array)
    predicted_class = np.argmax(prediction)
    class_labels = list(train_data.class_indices.keys())

    st.success(f"Predicted Rice Type: *{class_labels[predicted_class]}*")

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 789ms/step - accuracy: 0.2341 - loss: 3.3022 - val_accuracy: 0.3636 - val_loss: 8.3055
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step - accuracy: 0.2945 - loss: 6.1999 - val_accuracy: 0.2727 - val_loss: 4.9263
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step - accuracy: 0.2825 - loss: 3.9354 - val_accuracy: 0.3636 - val_loss: 3.0717
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.4252 - loss: 1.9834 - val_accuracy: 0.1818 - val_loss: 1.8813
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.3838 - loss: 1.5875 - val_accuracy: 0.3636 - val_loss: 1.8587
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step - accuracy: 0.5925 - loss: 1.2520 - val_accuracy: 0.4545 - val_loss: 2.0239
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 252ms/step - accuracy: 0.5276 - loss: 1.3767 - val_accuracy: 0.4545 - val_loss: 1.9293
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step - accuracy: 0.5890 - loss: 1.0976 - val_accuracy: 0.4545 - val_loss:

In [4]:
!pip install tensorflow

In [9]:
!pip install --upgrade numpy h5py


In [10]:
import numpy as np
import h5py
print("NumPy version:", np.__version__)
print("h5py version:", h5py.__version__)


NumPy version: 2.3.1
h5py version: 3.14.0


In [17]:
from tensorflow.keras import layers, models

num_classes = train_data.num_classes

model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 63504)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │       4,064,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,064,833 (15.51 MB)

 Trainable params: 4,064,833 (15.51 MB)

 Non-trainable params: 0 (0.00 B)